In this notebook I tried to get the location of the phone at every timestamp of the accelerometer measures for the train set. This is useful to get the position when wifi or ibeacon were measured. A similar model could be derived from the ones in this notebook to get the position during inference.

I used a state space model and the Kalman smoother to compute these locations. I tried with two models since I was a bit disappointed with the first one. Even with the second one I am still a bit frustrated. The model seems not to trust the accelerometer measures when I look at the optimized observation variances of the state space model. This makes me think I perhaps got visually correct results but with a poor modelling.

I would be glad to get any feedback to correct or improve the model.

___
# Packages

In [ ]:
import json
import os
from collections import OrderedDict
from dataclasses import dataclass

!pip install ahrs
from ahrs.filters import EKF
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import statsmodels.api as sm
from PIL import Image

___
# Parameters

In [ ]:
data_path = "/kaggle/input/indoor-location-navigation/"

___
# Functions

## Imported

In [ ]:
# copy from https://github.com/location-competition/indoor-location-competition-20/blob/master/io_f.py


@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, "r", encoding="utf-8") as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == "#":
            continue

        line_data = line_data.split("\t")

        if line_data[1] == "TYPE_ACCELEROMETER":
            acce.append(
                [
                    int(line_data[0]),
                    float(line_data[2]),
                    float(line_data[3]),
                    float(line_data[4]),
                ]
            )
            continue

        if line_data[1] == "TYPE_ACCELEROMETER_UNCALIBRATED":
            acce_uncali.append(
                [
                    int(line_data[0]),
                    float(line_data[2]),
                    float(line_data[3]),
                    float(line_data[4]),
                ]
            )
            continue

        if line_data[1] == "TYPE_GYROSCOPE":
            gyro.append(
                [
                    int(line_data[0]),
                    float(line_data[2]),
                    float(line_data[3]),
                    float(line_data[4]),
                ]
            )
            continue

        if line_data[1] == "TYPE_GYROSCOPE_UNCALIBRATED":
            gyro_uncali.append(
                [
                    int(line_data[0]),
                    float(line_data[2]),
                    float(line_data[3]),
                    float(line_data[4]),
                ]
            )
            continue

        if line_data[1] == "TYPE_MAGNETIC_FIELD":
            magn.append(
                [
                    int(line_data[0]),
                    float(line_data[2]),
                    float(line_data[3]),
                    float(line_data[4]),
                ]
            )
            continue

        if line_data[1] == "TYPE_MAGNETIC_FIELD_UNCALIBRATED":
            magn_uncali.append(
                [
                    int(line_data[0]),
                    float(line_data[2]),
                    float(line_data[3]),
                    float(line_data[4]),
                ]
            )
            continue

        if line_data[1] == "TYPE_ROTATION_VECTOR":
            ahrs.append(
                [
                    int(line_data[0]),
                    float(line_data[2]),
                    float(line_data[3]),
                    float(line_data[4]),
                ]
            )
            continue

        if line_data[1] == "TYPE_WIFI":
            sys_ts = line_data[0]
            ssid = line_data[
                2
            ]  # unique to a network, a WIFI network could be compose of many devices
            bssid = line_data[3]  # unique to a device
            rssi = line_data[4]  # strengh of signal
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == "TYPE_BEACON":
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            ibeacon_data = [ts, "_".join([uuid, major, minor]), rssi]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == "TYPE_WAYPOINT":
            waypoint.append(
                [int(line_data[0]), float(line_data[2]), float(line_data[3])]
            )

    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)

    #     print(acce.shape)
    #     print(acce_uncali.shape)
    #     print(gyro.shape)
    #     print(gyro_uncali.shape)
    #     print(magn.shape)
    #     print(magn_uncali.shape)
    #     print(ahrs.shape)
    #     print(wifi.shape)
    #     print(ibeacon.shape)
    #     print(waypoint.shape)

    return ReadData(
        acce,
        acce_uncali,
        gyro,
        gyro_uncali,
        magn,
        magn_uncali,
        ahrs,
        wifi,
        ibeacon,
        waypoint,
    )

## Imported and slightly modified

In [ ]:
def visualize_trajectory(
    trajectory,
    floor_plan_filename,
    width_meter,
    height_meter,
    title=None,
    mode="lines + markers + text",
    show=False,
    locs=None,
    directions=None
):
    """
    Copied from from https://github.com/location-competition/indoor-location-competition-20/blob/master/visualize_f.py

    """
    fig = go.Figure()

    # add trajectory
    size_list = [6] * trajectory.shape[0]
    size_list[0] = 10
    size_list[-1] = 10

    color_list = ["rgba(4, 174, 4, 0.5)"] * trajectory.shape[0]
    color_list[0] = "rgba(12, 5, 235, 1)"
    color_list[-1] = "rgba(235, 5, 5, 1)"

    position_count = {}
    text_list = []
    for i in range(trajectory.shape[0]):
        if str(trajectory[i]) in position_count:
            position_count[str(trajectory[i])] += 1
        else:
            position_count[str(trajectory[i])] = 0
        text_list.append("        " * position_count[str(trajectory[i])] + f"{i}")
    text_list[0] = "Start 0"
    text_list[-1] = f"End {trajectory.shape[0] - 1}"

    fig.add_trace(
        go.Scattergl(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            mode=mode,
            marker=dict(size=size_list, color=color_list),
            line=dict(shape="linear", color="lightgrey", width=3, dash="dash"),
            text=text_list,
            textposition="top center",
            name="trajectory",
        )
    )

    if isinstance(locs, pd.DataFrame):
        fig.add_trace(go.Scatter(x=locs["x"], y=locs["y"], mode="lines", name="position"))
        
    if isinstance(directions, list):
        for direction in directions:
            fig.add_trace(
                go.Scatter(
                    x=[direction["x"], direction["x"] + direction["d_x"]],
                    y=[direction["y"], direction["y"] + direction["d_y"]],
                    mode="lines",
                    **direction["kwargs"],
                )
            )

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(
        images=[
            go.layout.Image(
                source=floor_plan,
                xref="x",
                yref="y",
                x=0,
                y=height_meter,
                sizex=width_meter,
                sizey=height_meter,
                sizing="contain",
                opacity=1,
                layer="below",
            )
        ]
    )

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(
        autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1
    )
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=800,
        height=800 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig


def visualize_train_trajectory(path, **kwargs):
    """
    Edited from
    https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis
    """
    _id, floor = path.split("/")[:2]

    train_floor_data = read_data_file(os.path.join(data_path, "train", path))
    with open(os.path.join(data_path, "metadata", _id, floor, "floor_info.json")) as f:
        train_floor_info = json.load(f)

    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3],
        os.path.join(data_path, "metadata", _id, floor, "floor_image.png"),
        train_floor_info["map_info"]["width"],
        train_floor_info["map_info"]["height"],
        f"Visualization of {path}",
        **kwargs
    )

## Own ones

### To get data

In [ ]:
def build_meas_df(data):
    acc = pd.DataFrame(data.acce, columns=["timestamp", "acc_x", "acc_y", "acc_z"])
    gyro = pd.DataFrame(data.gyro, columns=["timestamp", "gyro_x", "gyro_y", "gyro_z"])
    magn = pd.DataFrame(data.magn, columns=["timestamp", "magn_x", "magn_y", "magn_z"])
    ahrs = pd.DataFrame(data.ahrs, columns=["timestamp", "ahrs_x", "ahrs_y", "ahrs_z"])
    df = (
        acc.merge(gyro, on="timestamp")
        .merge(magn, on="timestamp")
        .merge(ahrs, on="timestamp")
    )
    df["timestamp"] = df["timestamp"].astype(int)
    return df


def build_waypoint_df(data):
    out = pd.DataFrame(
        data.waypoint,
        columns=[
            "timestamp",
            "x",
            "y",
        ],
    )
    out["timestamp"] = out["timestamp"].astype(int)
    out["x"] = out["x"].astype(float)
    out["y"] = out["y"].astype(float)
    return out

### Location models

#### Model 1

State space model. The x and y position can then be computed by means of Kalman smoother.

Between two time steps I consider that the acceleration is constant, hence the dynamic equations below. I also assumed that the measurement error variances of the accelerometers may differ from an axis to another. I finally assumed that the variances of the acceleration transition could also be different from an axis to another.

The states are:
- positions x and y in the coordinate system of the buildings
- speeds along x and y in the coordinate system of the buildings
- accelerations along x, y and z in the coordinate system of the phone
- bias along x, y and z of the phone accelerometers

I arbitrarily the following order:
[x, v_x, y, v_y, acc_x, acc_y, acc_z, bias_x, bias_y, bias_z]

The measures are:
- positions x and y sometimes
- accelerations + bias from the phone sensors at every time steps on the 3 axes

In [ ]:
class Model1(sm.tsa.statespace.MLEModel):
    def __init__(
        self,
        acc_x_t: np.array,
        acc_y_t: np.array,
        acc_z_t: np.array,
        x_t: np.array,
        y_t: np.array,
        r_x_x: np.array,
        r_x_y: np.array,
        r_x_z: np.array,
        r_y_x: np.array,
        r_y_y: np.array,
        r_y_z: np.array,
        delta_t: float,
    ):
        super(Model1, self).__init__(
            endog=np.c_[x_t, y_t, acc_x_t, acc_y_t, acc_z_t],
            k_states=10,
            k_posdef=3,
            initialization="approximate_diffuse",
        )

        self.ssm["design"] = np.zeros((self.k_endog, self.k_states, self.nobs))
        self.ssm["design", 0, 0, :] = 1  # x
        self.ssm["design", 1, 2, :] = 1  # y
        self.ssm["design", 2, 4, :] = 1  # acc x phone
        self.ssm["design", 2, 7, :] = 1  # bias x phone
        self.ssm["design", 3, 5, :] = 1  # acc y phone
        self.ssm["design", 3, 8, :] = 1  # bias y phone
        self.ssm["design", 4, 6, :] = 1  # acc z phone
        self.ssm["design", 4, 9, :] = 1  # bias z phone

        self.ssm["transition"] = np.zeros((self.k_states, self.k_states, self.nobs))
        self.ssm["selection"] = np.zeros((self.k_states, 3, self.nobs))
        # x(n+1)
        self.ssm["transition", 0, 0, :] = 1
        self.ssm["transition", 0, 1, :] = delta_t
        self.ssm["selection", 0, 0, :] = r_x_x * delta_t ** 2 / 2
        self.ssm["selection", 0, 1, :] = r_x_y * delta_t ** 2 / 2
        self.ssm["selection", 0, 2, :] = r_x_z * delta_t ** 2 / 2
        # v_x(n+1)
        self.ssm["transition", 1, 1, :] = 1
        self.ssm["selection", 1, 0, :] = r_x_x * delta_t
        self.ssm["selection", 1, 1, :] = r_x_y * delta_t
        self.ssm["selection", 1, 2, :] = r_x_z * delta_t
        # y(n+1)
        self.ssm["transition", 2, 2, :] = 1
        self.ssm["transition", 2, 3, :] = delta_t
        self.ssm["selection", 2, 0, :] = r_y_x * delta_t ** 2 / 2
        self.ssm["selection", 2, 1, :] = r_y_y * delta_t ** 2 / 2
        self.ssm["selection", 2, 2, :] = r_y_z * delta_t ** 2 / 2
        # v_y(n+1)
        self.ssm["transition", 3, 3, :] = 1
        self.ssm["selection", 3, 0, :] = r_y_x * delta_t
        self.ssm["selection", 3, 1, :] = r_y_y * delta_t
        self.ssm["selection", 3, 2, :] = r_y_z * delta_t
        # acc_x(n+1)
        self.ssm["selection", 4, 0, :] = 1
        # acc_y(n+1)
        self.ssm["selection", 5, 1, :] = 1
        # acc_z(n+1)
        self.ssm["selection", 6, 2, :] = 1
        # bias_x(n+1)
        self.ssm["transition", 7, 7, :] = 1
        # bias_x(n+1)
        self.ssm["transition", 8, 8, :] = 1
        # bias_x(n+1)
        self.ssm["transition", 9, 9, :] = 1

        self.position_dict = OrderedDict(
            var_acc_x_dyn=1,
            var_acc_y_dyn=2,
            var_acc_z_dyn=3,
            var_acc_x_meas=4,
            var_acc_y_meas=5,
            var_acc_z_meas=6,
        )
        self.initial_values = starting_values
        self.positive_parameters = slice(0, 6)

    @property
    def param_names(self):
        return list(self.position_dict.keys())

    @property
    def start_params(self):
        params = np.r_[
            self.initial_values["var.acc_x_dyn"],
            self.initial_values["var.acc_y_dyn"],
            self.initial_values["var.acc_z_dyn"],
            self.initial_values["var.acc_x_meas"],
            self.initial_values["var.acc_y_meas"],
            self.initial_values["var.acc_z_meas"],
        ]
        return params

    def transform_params(self, unconstrained):
        """
        We constraint all parameters to be positive,
        because they are variances
        """
        constrained = unconstrained.copy()
        constrained[self.positive_parameters] = (
            constrained[self.positive_parameters] ** 2
        )
        return constrained

    def untransform_params(self, constrained):
        """
        Need to unstransform all the parameters you transformed
        in the `transform_params` function
        """
        unconstrained = constrained.copy()
        unconstrained[self.positive_parameters] = (
            unconstrained[self.positive_parameters] ** 0.5
        )
        return unconstrained

    def update(self, params, **kwargs):
        params = super(Model1, self).update(params, **kwargs)
        self["obs_cov", 2, 2] = params[3]
        self["obs_cov", 3, 3] = params[4]
        self["obs_cov", 4, 4] = params[5]
        self["state_cov", 0, 0] = params[0]
        self["state_cov", 1, 1] = params[1]
        self["state_cov", 2, 2] = params[2]

#### Model 2

The model 2 is very similar to the model 1. I was not satisfied with the results obtained with the first model. I observed that the vertical accelerometer seems to indicate that the phone carrier was steady when the positions were stored. The vertical acceleration is almost constant which indicates that there is no step. For these two reasons I tried to add the speeds to the observations when locations x and y were measured. I arbitrarily chose to set the speed at these points to 0, which I would say is a strong assumption. 

The states are the same:
- positions x and y in the coordinate system of the buildings
- speeds along x and y in the coordinate system of the buildings
- accelerations along x, y and z in the coordinate system of the phone
- bias along x, y and z of the phone accelerometers

The order is the same:
[x, v_x, y, v_y, acc_x, acc_y, acc_z, bias_x, bias_y, bias_z]

The measures are now:
- positions x and y sometimes
- accelerations + bias from the phone sensors at every time steps on the 3 axes
- speeds along x and y sometimes

In [ ]:
class Model2(sm.tsa.statespace.MLEModel):
    def __init__(
        self,
        acc_x_t: np.array,
        acc_y_t: np.array,
        acc_z_t: np.array,
        x_t: np.array,
        y_t: np.array,
        r_x_x: np.array,
        r_x_y: np.array,
        r_x_z: np.array,
        r_y_x: np.array,
        r_y_y: np.array,
        r_y_z: np.array,
        delta_t: float,
        v_x: np.array,
        v_y:np.array
    ):
        super(Model2, self).__init__(
            endog=np.c_[x_t, y_t, acc_x_t, acc_y_t, acc_z_t, v_x, v_y],
            k_states=10,
            k_posdef=3,
            initialization="approximate_diffuse",
        )

        self.ssm["design"] = np.zeros((self.k_endog, self.k_states, self.nobs))
        self.ssm["design", 0, 0, :] = 1  # x
        self.ssm["design", 1, 2, :] = 1  # y
        self.ssm["design", 2, 4, :] = 1  # acc x phone
        self.ssm["design", 2, 7, :] = 1  # bias x phone
        self.ssm["design", 3, 5, :] = 1  # acc y phone
        self.ssm["design", 3, 8, :] = 1  # bias y phone
        self.ssm["design", 4, 6, :] = 1  # acc z phone
        self.ssm["design", 4, 9, :] = 1  # bias z phone
        self.ssm["design", 5, 1, :] = 1  # v_x
        self.ssm["design", 6, 3, :] = 1  # v_y

        self.ssm["transition"] = np.zeros((self.k_states, self.k_states, self.nobs))
        self.ssm["selection"] = np.zeros((self.k_states, 3, self.nobs))
        # x(n+1)
        self.ssm["transition", 0, 0, :] = 1
        self.ssm["transition", 0, 1, :] = delta_t
        self.ssm["selection", 0, 0, :] = r_x_x * delta_t ** 2 / 2
        self.ssm["selection", 0, 1, :] = r_x_y * delta_t ** 2 / 2
        self.ssm["selection", 0, 2, :] = r_x_z * delta_t ** 2 / 2
        # v_x(n+1)
        self.ssm["transition", 1, 1, :] = 1
        self.ssm["selection", 1, 0, :] = r_x_x * delta_t
        self.ssm["selection", 1, 1, :] = r_x_y * delta_t
        self.ssm["selection", 1, 2, :] = r_x_z * delta_t
        # y(n+1)
        self.ssm["transition", 2, 2, :] = 1
        self.ssm["transition", 2, 3, :] = delta_t
        self.ssm["selection", 2, 0, :] = r_y_x * delta_t ** 2 / 2
        self.ssm["selection", 2, 1, :] = r_y_y * delta_t ** 2 / 2
        self.ssm["selection", 2, 2, :] = r_y_z * delta_t ** 2 / 2
        # v_y(n+1)
        self.ssm["transition", 3, 3, :] = 1
        self.ssm["selection", 3, 0, :] = r_y_x * delta_t
        self.ssm["selection", 3, 1, :] = r_y_y * delta_t
        self.ssm["selection", 3, 2, :] = r_y_z * delta_t
        # acc_x(n+1)
        self.ssm["selection", 4, 0, :] = 1
        # acc_y(n+1)
        self.ssm["selection", 5, 1, :] = 1
        # acc_z(n+1)
        self.ssm["selection", 6, 2, :] = 1
        # bias_x(n+1)
        self.ssm["transition", 7, 7, :] = 1
        # bias_x(n+1)
        self.ssm["transition", 8, 8, :] = 1
        # bias_x(n+1)
        self.ssm["transition", 9, 9, :] = 1

        self.position_dict = OrderedDict(
            var_acc_x_dyn=1,
            var_acc_y_dyn=2,
            var_acc_z_dyn=3,
            var_acc_x_meas=4,
            var_acc_y_meas=5,
            var_acc_z_meas=6,
        )
        self.initial_values = starting_values
        self.positive_parameters = slice(0, 6)

    @property
    def param_names(self):
        return list(self.position_dict.keys())

    @property
    def start_params(self):
        params = np.r_[
            self.initial_values["var.acc_x_dyn"],
            self.initial_values["var.acc_y_dyn"],
            self.initial_values["var.acc_z_dyn"],
            self.initial_values["var.acc_x_meas"],
            self.initial_values["var.acc_y_meas"],
            self.initial_values["var.acc_z_meas"],
        ]
        return params

    def transform_params(self, unconstrained):
        """
        We constraint all parameters to be positive,
        because they are variances
        """
        constrained = unconstrained.copy()
        constrained[self.positive_parameters] = (
            constrained[self.positive_parameters] ** 2
        )
        return constrained

    def untransform_params(self, constrained):
        """
        Need to unstransform all the parameters you transformed
        in the `transform_params` function
        """
        unconstrained = constrained.copy()
        unconstrained[self.positive_parameters] = (
            unconstrained[self.positive_parameters] ** 0.5
        )
        return unconstrained

    def update(self, params, **kwargs):
        params = super(Model2, self).update(params, **kwargs)
        self["obs_cov", 2, 2] = params[3]
        self["obs_cov", 3, 3] = params[4]
        self["obs_cov", 4, 4] = params[5]
        self["state_cov", 0, 0] = params[0]
        self["state_cov", 1, 1] = params[1]
        self["state_cov", 2, 2] = params[2]

> ___
# Compute location at every time steps

## Chosen file

In [ ]:
building = "5a0546857ecc773753327266"
floor = "F4"
path = "5d11dc28ffe23f0008604f67"
building_floor_path = building + "/" + floor + "/" + path + ".txt"

## Get data

In [ ]:
data = read_data_file(os.path.join(data_path, "train", building_floor_path))
dynamic_df = build_meas_df(data)
dynamic_df["path"] = path
waypoint_df = build_waypoint_df(data)
waypoint_df["path"] = path

## Add waypoint to dynamic

In [ ]:
dynamic_with_waypoint = dynamic_df.merge(
    waypoint_df, on="path", suffixes=("_dynamic", "_waypoint")
)
dynamic_with_waypoint["timestamp_diff"] = (
    dynamic_with_waypoint["timestamp_dynamic"] - dynamic_with_waypoint["timestamp_waypoint"]
).abs()
dynamic_with_waypoint.sort_values(["timestamp_waypoint", "timestamp_diff"], inplace=True)
dynamic_with_waypoint.drop_duplicates(["timestamp_waypoint"], inplace=True)
dynamic_with_waypoint = dynamic_with_waypoint[["timestamp_dynamic", "x", "y"]]
dynamic_with_waypoint.rename(columns={"timestamp_dynamic": "timestamp"}, inplace=True)
dynamic_df_with_waypoint = dynamic_df.merge(dynamic_with_waypoint, how="left")

## Get rotation matrix (only 6 components are enough in the model)

The rotation vector of the phone is already given in the provided data through an unit quaternion at every timestamp. For a sanity check, a comparison has been carried out between the existing rotation vector data and the estimation of the phone orientation with an extended Kalman filter provided by the library [ahrs](http://ahrs.readthedocs.io/en/latest/filters/ekf.html).

### Provided data

In [ ]:
# Add quaternion first component
first_comp_quarternion = (
    1
    - dynamic_df_with_waypoint["ahrs_x"] ** 2
    - dynamic_df_with_waypoint["ahrs_y"] ** 2
    - dynamic_df_with_waypoint["ahrs_z"] ** 2
)
first_comp_quarternion.loc[first_comp_quarternion < 0] = 0
dynamic_df_with_waypoint["quaternion_cos"] = first_comp_quarternion.apply(np.sqrt)

# Add rotation matrix components
dynamic_df_with_waypoint["r_x_x"] = (
    dynamic_df_with_waypoint["quaternion_cos"] ** 2
    + dynamic_df_with_waypoint["ahrs_x"] ** 2
    - dynamic_df_with_waypoint["ahrs_y"] ** 2
    - dynamic_df_with_waypoint["ahrs_z"] ** 2
)
dynamic_df_with_waypoint["r_x_y"] = (
    2 * dynamic_df_with_waypoint["ahrs_x"] * dynamic_df_with_waypoint["ahrs_y"]
    - 2
    * dynamic_df_with_waypoint["quaternion_cos"]
    * dynamic_df_with_waypoint["ahrs_z"]
)
dynamic_df_with_waypoint["r_x_z"] = (
    2 * dynamic_df_with_waypoint["quaternion_cos"] * dynamic_df_with_waypoint["ahrs_y"]
    + 2 * dynamic_df_with_waypoint["ahrs_x"] * dynamic_df_with_waypoint["ahrs_z"]
)

dynamic_df_with_waypoint["r_y_x"] = (
    2 * dynamic_df_with_waypoint["quaternion_cos"] * dynamic_df_with_waypoint["ahrs_z"]
    + 2 * dynamic_df_with_waypoint["ahrs_x"] * dynamic_df_with_waypoint["ahrs_y"]
)
dynamic_df_with_waypoint["r_y_y"] = (
    dynamic_df_with_waypoint["quaternion_cos"] ** 2
    - dynamic_df_with_waypoint["ahrs_x"] ** 2
    + dynamic_df_with_waypoint["ahrs_y"] ** 2
    - dynamic_df_with_waypoint["ahrs_z"] ** 2
)
dynamic_df_with_waypoint["r_y_z"] = (
    2 * dynamic_df_with_waypoint["ahrs_y"] * dynamic_df_with_waypoint["ahrs_z"]
    - 2
    * dynamic_df_with_waypoint["quaternion_cos"]
    * dynamic_df_with_waypoint["ahrs_x"]
)

dynamic_df_with_waypoint["r_z_x"] = (
    2 * dynamic_df_with_waypoint["ahrs_x"] * dynamic_df_with_waypoint["ahrs_z"]
    - 2
    * dynamic_df_with_waypoint["quaternion_cos"]
    * dynamic_df_with_waypoint["ahrs_y"]
)
dynamic_df_with_waypoint["r_z_y"] = (
    2 * dynamic_df_with_waypoint["quaternion_cos"] * dynamic_df_with_waypoint["ahrs_x"]
    + 2 * dynamic_df_with_waypoint["ahrs_y"] * dynamic_df_with_waypoint["ahrs_z"]
)
dynamic_df_with_waypoint["r_z_z"] = (
    dynamic_df_with_waypoint["quaternion_cos"] ** 2
    - dynamic_df_with_waypoint["ahrs_x"] ** 2
    - dynamic_df_with_waypoint["ahrs_y"] ** 2
    + dynamic_df_with_waypoint["ahrs_z"] ** 2
)

### By means of an extend Kalman filter and gyroscope, acceleration and magnetic field measures

In [ ]:
ekf = EKF(
    gyr=dynamic_df_with_waypoint[["gyro_x", "gyro_y", "gyro_z"]].values,
    acc=dynamic_df_with_waypoint[["acc_x", "acc_y", "acc_z"]].values,
    mag=dynamic_df_with_waypoint[["magn_x", "magn_y", "magn_z"]].values,
    frequency=1/dynamic_df_with_waypoint["timestamp"].diff().mean()*1e3,
    frame="ENU",
)

Q = ekf.Q
q0 = Q[:, 0]
q1 = Q[:, 1]
q2 = Q[:, 2]
q3 = Q[:, 3]

dynamic_df_with_waypoint["r_x_x_2"] = q0 ** 2 + q1 ** 2 - q2 ** 2 - q3 ** 2
dynamic_df_with_waypoint["r_x_y_2"] = 2 * (q1 * q2 - q0 * q3)
dynamic_df_with_waypoint["r_x_z_2"] = 2 * (q1 * q3 + q0 * q2)

dynamic_df_with_waypoint["r_y_x_2"] = 2 * (q1 * q2 + q0 * q3)
dynamic_df_with_waypoint["r_y_y_2"] = q0 ** 2 - q1 ** 2 + q2 ** 2 - q3 ** 2
dynamic_df_with_waypoint["r_y_z_2"] = 2 * (q2 * q3 - q0 * q1)

### Get directions at all waypoint measures

As explained in the data description, the phone is held horizontally with its y axis pointing towards where the person walks. I then expect the phone y axis to be tangent to the path. 

In [ ]:
arrows = dynamic_df_with_waypoint.loc[~dynamic_df_with_waypoint["x"].isnull()]

directions = []
showlegend = True
for _, row in arrows.iterrows():
    # direction from provided data
    direction = {
        "x": row["x"],
        "y": row["y"],
        "d_x": row["r_x_y"],
        "d_y": row["r_y_y"],
        "kwargs": {
            "line": {"color": "red"},
            "legendgroup": 1,
            "showlegend": showlegend,
            "name": "provided data",
        },
    }
    directions.append(direction)

    # direction from the EKF results
    direction = {
        "x": row["x"],
        "y": row["y"],
        "d_x": row["r_x_y_2"],
        "d_y": row["r_y_y_2"],
        "kwargs": {
            "line": {"color": "blue"},
            "legendgroup": 2,
            "showlegend": showlegend,
            "name": "computed with EKF",
        },
    }
    directions.append(direction)
    showlegend = False

In [ ]:
visualize_train_trajectory(building_floor_path, directions=directions)

The directions are not perfectly superimposed. This may be due to different parameters of the extended Kalman filter used by the Android phone to get the rotation vector or maybe because something else is used. However, even though they are not superimposed, they are very similar which is comforting.

## Get data for models

In [ ]:
r_x_x = dynamic_df_with_waypoint["r_x_x"].to_numpy()
r_x_y = dynamic_df_with_waypoint["r_x_y"].to_numpy()
r_x_z = dynamic_df_with_waypoint["r_x_z"].to_numpy()

r_y_x = dynamic_df_with_waypoint["r_y_x"].to_numpy()
r_y_y = dynamic_df_with_waypoint["r_y_y"].to_numpy()
r_y_z = dynamic_df_with_waypoint["r_y_z"].to_numpy()

r_z_x = dynamic_df_with_waypoint["r_z_x"].to_numpy()
r_z_y = dynamic_df_with_waypoint["r_z_y"].to_numpy()
r_z_z = dynamic_df_with_waypoint["r_z_z"].to_numpy()

# Get acceleration without gravity. A steady accelerometer along the vertical axis and towards up measures +1g.
g = 9.81
acc_x_t = (dynamic_df_with_waypoint["acc_x"] - r_z_x * g).to_numpy()
acc_y_t = (dynamic_df_with_waypoint["acc_y"] - r_z_y * g).to_numpy()
acc_z_t = (dynamic_df_with_waypoint["acc_z"] - r_z_z * g).to_numpy()

x_t = dynamic_df_with_waypoint["x"].to_numpy()
y_t = dynamic_df_with_waypoint["y"].to_numpy()

v_x = (dynamic_df_with_waypoint["x"] * 0).to_numpy()
v_y = (dynamic_df_with_waypoint["x"] * 0).to_numpy()

delta_t = dynamic_df_with_waypoint["timestamp"].diff().mean() / 1e3

## Model 1

In [ ]:
starting_values = {
    "var.acc_x_dyn": 1,
    "var.acc_y_dyn": 1,
    "var.acc_z_dyn": 1,
    "var.acc_x_meas": 0.1,
    "var.acc_y_meas": 0.1,
    "var.acc_z_meas": 0.1,
}

model_1 = Model1(
    acc_x_t,
    acc_y_t,
    acc_z_t,
    x_t,
    y_t,
    r_x_x,
    r_x_y,
    r_x_z,
    r_y_x,
    r_y_y,
    r_y_z,
    delta_t,
)
res_model_1 = model_1.fit(maxiter=100)

In [ ]:
res_model_1.summary()

In [ ]:
dynamic_df_with_waypoint["x_smoothed_model_1"] = res_model_1.smoothed_state[0, :]
dynamic_df_with_waypoint["y_smoothed_model_1"] = res_model_1.smoothed_state[2, :]

In [ ]:
fig = plot.figure(figsize=(16, 16))

ss = pd.DataFrame(
    res_model_1.smoothed_state.T,
    columns=[
        "x",
        "v_x",
        "y",
        "v_y",
        "acc_x",
        "acc_y",
        "acc_z",
        "bias_acc_x",
        "bias_acc_y",
        "bias_acc_z",
    ],
    index=dynamic_df_with_waypoint["timestamp"],
)

ax = fig.add_subplot(2, 1, 1)
ax.plot(ss["x"], label="Smoothed estimate x")
ax.plot(
    waypoint_df["timestamp"],
    waypoint_df["x"],
    linestyle="",
    marker="+",
    label="Actual x"
)
ax.set_xlabel("timestamp [s]")
ax.set_ylabel("position [m]")
ax.legend()

ax = fig.add_subplot(2, 1, 2)
ax.plot(ss["y"], label="Smoothed estimate y")
ax.plot(
    waypoint_df["timestamp"],
    waypoint_df["y"],
    linestyle="",
    marker="+",
    label="Actual y"
)
ax.set_xlabel("timestamp [s]")
ax.set_ylabel("position [m]")
ax.legend()

plot.show()

In [ ]:
locs = dynamic_df_with_waypoint[["x_smoothed_model_1", "y_smoothed_model_1"]]
locs.rename(columns={"x_smoothed_model_1": "x", "y_smoothed_model_1": "y"}, inplace=True)
visualize_train_trajectory(building_floor_path, locs=locs)

Some positions are out of the path which makes me try to improve the model.

## Model 2

In [ ]:
starting_values = {
    "var.acc_x_dyn": 1,
    "var.acc_y_dyn": 1,
    "var.acc_z_dyn": 1,
    "var.acc_x_meas": 0.1,
    "var.acc_y_meas": 0.1,
    "var.acc_z_meas": 0.1,
}

model_2 = Model2(
    acc_x_t,
    acc_y_t,
    acc_z_t,
    x_t,
    y_t,
    r_x_x,
    r_x_y,
    r_x_z,
    r_y_x,
    r_y_y,
    r_y_z,
    delta_t,
    v_x,
    v_y
)
res_model_2 = model_2.fit(maxiter=100)

In [ ]:
res_model_2.summary()

In [ ]:
dynamic_df_with_waypoint["x_smoothed_model_2"] = res_model_2.smoothed_state[0, :]
dynamic_df_with_waypoint["y_smoothed_model_2"] = res_model_2.smoothed_state[2, :]

In [ ]:
fig = plot.figure(figsize=(16, 16))

ss = pd.DataFrame(
    res_model_2.smoothed_state.T,
    columns=[
        "x",
        "v_x",
        "y",
        "v_y",
        "acc_x",
        "acc_y",
        "acc_z",
        "bias_acc_x",
        "bias_acc_y",
        "bias_acc_z",
    ],
    index=dynamic_df_with_waypoint["timestamp"],
)

ax = fig.add_subplot(2, 1, 1)
ax.plot(ss["x"], label="Smoothed estimate x")
ax.plot(
    waypoint_df["timestamp"],
    waypoint_df["x"],
    linestyle="",
    marker="+",
    label="Actual x"
)
ax.set_xlabel("timestamp [s]")
ax.set_ylabel("position [m]")
ax.legend()

ax = fig.add_subplot(2, 1, 2)
ax.plot(ss["y"], label="Smoothed estimate y")
ax.plot(
    waypoint_df["timestamp"],
    waypoint_df["y"],
    linestyle="",
    marker="+",
    label="Actual y"
)
ax.set_xlabel("timestamp [s]")
ax.set_ylabel("position [m]")
ax.legend()

plot.show()

In [ ]:
locs = dynamic_df_with_waypoint[["x_smoothed_model_2", "y_smoothed_model_2"]]
locs.rename(columns={"x_smoothed_model_2": "x", "y_smoothed_model_2": "y"}, inplace=True)
visualize_train_trajectory(building_floor_path, locs=locs)

___
# Outlook
I am thinking of a state space model during inference to compute the position of the phone at every timestamp. During inference, there is no waypoint measure. Location could be estimated when wifi or ibeacon are measured. Contrary to for the train set, these measures would not be 100% accurate. Then the observation covariance matrix would be changed with additional non zero values for the variances of these measured. I don't know yet how to evaluate these values. Regarding the parameters optimized with the train set, I would fix them during inference using for instance means of the values obtained with all the paths.